# Funciones
En este notebook vemos las funciones vista en clase que nos servirán de ayuda para cuando desarrollemos aquellas que nos pide el enunciado.

In [1]:
import pandas as pd
import mysql.connector as connection
from google.cloud import bigquery
from dotenv import load_dotenv
import os
load_dotenv()

True

Hacemos una funcion la cual me devolverá los "order_customer_id" que cumplen ciertos requisitos:

In [2]:
def listaOrders(order_customer_id):
    mydb = connection.connect(
        host= os.getenv('DB_HOST'), 
        user= os.getenv('DB_USER'), 
        passwd= os.getenv('DB_PASS'), 
        database=os.getenv('DB_DB') 
    )

    sql = f"SELECT * FROM retail_db.orders WHERE order_customer_id ={order_customer_id}"
    df = pd.read_sql(sql, mydb)
    mydb.close()
    return df.to_json(orient="records") # como lo vamos a aplicar en cloud functions el formato que necesitamos es json (http) sino solo retornamos df

In [3]:
listaOrders(2000)

C:\Users\GGudino\AppData\Local\Temp\ipykernel_948\2115503731.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, mydb)


'[{"order_id":28408,"order_date":1389830400000,"order_customer_id":2000,"order_status":"PROCESSING"},{"order_id":29016,"order_date":1390176000000,"order_customer_id":2000,"order_status":"PENDING_PAYMENT"},{"order_id":31989,"order_date":1391731200000,"order_customer_id":2000,"order_status":"COMPLETE"},{"order_id":63365,"order_date":1393027200000,"order_customer_id":2000,"order_status":"COMPLETE"},{"order_id":68481,"order_date":1400630400000,"order_customer_id":2000,"order_status":"PENDING_PAYMENT"},{"order_id":68884,"order_date":1689984000000,"order_customer_id":2000,"order_status":"PENDING"}]'

### Cloud Functions
Si a esta función la llevamos a Cloud Functions en GCP deberemos:
1. Hacer click en Crear función
2. Elegir el entorno de 1era generación para generar menos costos
3. Darle un nombre a la función
4. En AUTENTICACIÓN seleccionamos "Permitir invocaciones sin autenticar"
5. Ya en la parte de "Código" elegiremos el entorno de ejecución, que en este caso será Python 3.11
6. Donde indica punto de entrada deberemos poner el nombre de nuestra función
7. En el espacio que tenemos, cargamos el script de la función. Tenemos que tener en cuenta que en este ambiente las funciones no aceptan argumentos sino "request" (que es lo que pondremos en el http luego de un "?", cuando consultemos a la función). Luego dentro de la función deberemos definir los argumentos del "request". Ejemplo:
```python
def listaOrders(request):
    request_args = request.args
    order_customer_id = request_args["order_customer_id"] # podemos poner otro nombre dentro de request_args ya que es como identificar al argumento en la url.
    mydb = connection.connect(
        host= 'DB_HOST', 
        user= 'DB_USER', 
        passwd= 'DB_PASS', 
        database= 'DB_DB'
    )

    sql = f"SELECT * FROM retail_db.orders WHERE order_customer_id ={order_customer_id}"
    df = pd.read_sql(sql, mydb)
    mydb.close()
    return df.to_json(orient="records")
```
Consultando luego la Cloud Function:
```
www.urldeejemplo.com/listaOrders?order_customer_id=1117
```
8. Luego en el archivo requirements.txt de Cloud Functions ponemos (no se usarán todas):
google-cloud
google.cloud.storage
mysql-connector-python
google-api-python-client
google-cloud-bigquery
google-cloud-bigquery-storage
SQLAlchemy
pyarrow
pandas
9. En "Permisos" habilitar el uso para que cualquiera pueda invocar la función. 
    - Vamos a "Otorgar acceso"
    - Donde nos da la opción de completar "Principales nuevas" ponemos "allUsers"
    - Rol: "Invocador de Cloud Functions"

### Insert
Ahora vamos a crear una función que inserte datos en nuestra base de datos

In [4]:
from datetime import date

def insertarOrdenes(order_customer_id, order_status):
    mydb = connection.connect(
        host= os.getenv('DB_HOST'), 
        user= os.getenv('DB_USER'), 
        passwd= os.getenv('DB_PASS'), 
        database=os.getenv('DB_DB') 
    )

    order_date = date.today()

    mycursor = mydb.cursor() # necesario para interactuar con la BD cuando queremos realizar "inserts"

    #query = f"INSERT INTO retail_db.orders(order_date, order_customer_id, order_status) VALUES ({order_date},{order_customer_id}, {order_status})" # abajo alternativa para no usar el f
    query = "INSERT INTO retail_db.orders(order_date, order_customer_id, order_status) VALUES (%s,%s, %s)"
    val = (order_date, order_customer_id, order_status)
    mycursor.execute(query,val)
    mydb.commit()
    mydb.close()
    return "¡Datos cargados!"

In [5]:
insertarOrdenes(2000, "PENDING")

'¡Datos cargados!'